# Iris 4: Iris-Dataset - Das Neuronale Netz

Beim Iris-Datensatz haben wir gesehen, dass die zusammengehörenden Datensätze im vierdimensionalen Raum nahe beieinanderliegen, also __Cluster__ bilden. Durch einfaches Betrachten verschiedener Darstellungen der Daten konnten wir diese Cluster leicht trennen. Der "natürliche" Ansatz, dies zu autimatisieren, war die Methode des __k-means__: Wir suchten k = 3 verscheidenen Mittelwerte, um die sich diese Cluster bilden.

Obwohl das Problem dadruch hinreichend gut gelöst ist, wollen wir nun versuchen, das Problem mit einem einfachen neuronalen netzwerk zu lösen. Diese Lösung ist zwar schon fast überdimensioniert, aber da das Problem noch übersichtlich ist, können wir die Wirkungsweise neuronaler Netzwerke hier gut erkennen, bisvor wir zu größeren Problemen (was Daten-Dimension und -Volumen anbetrifft) übergehen.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import ki_kurs.iris as ir
import pprint as pp

## 1. Schritt: Erstellung von Trainings- und Testdaten aus der Iris-Datenbank

Wir lesen zunächst die Daten aus der Iris-Datenbank ein und teilen diese dann in eine Menge von Trainingsdaten auf, mit denen das Netzwerk trainiert wird, und eine Menge von Testdaten, mit denen wir dann das Ergebnis überprüfen.

Da wir eine Art generisches KNN definieren wollen, skalieren wir die Werte unserer Iris-DB auf den Bereich $[0,1]$. Dies haben wir in der Funktion _tt_daten_ bereits erledigt.

In [2]:
iris = ir.IrisDataSet("daten/iris.csv")
trainings_daten, test_daten = iris.tt_daten(100)

## Schritt 2: Programmierung eines dreischichtigen Netzwerks

In [3]:
# Ein generisches Neuronales Netzwerk mit einer Eingabeschicht, einer versteckten Schicht und einer Ausgabeschicht.
    
class NeuralNetwork:
    def __init__(self, *nodes: list[int]) -> None:
        ''' Setzen der Parameter des Neuronalen Netzwerks. Gewichte werden zufällig erzeugt. '''
        self.inodes, self.hnodes, self.onodes = nodes

        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        self.transfer = lambda x: 1/(1 + np.exp(-x)) # Die Sigmoid-Funktion
        pass

    def train(self, inputs_list: np.ndarray, targets_list: np.ndarray,lr: float = 0.2) -> None :
        ''' Training des Neuronalen Netzwerks '''
   
        inputs =  np.transpose(np.array(inputs_list, ndmin=2))
        targets = np.transpose(np.array(targets_list, ndmin=2))

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.transfer(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.transfer(final_inputs)

        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # Backpropagation
        self.who += lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        self.wih += lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
    
        pass

    def query(self, inputs_list: np.ndarray):
        ''' Abfrage des Neuronalen Netzwerks '''
        inputs = np.array(inputs_list, ndmin=2).T
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.transfer(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.transfer(final_inputs)
        return np.concatenate(final_outputs).ravel()

    def result(self, inputs: np.ndarray) -> int:
        return np.argmax(self.query(inputs))

    def save(self,file: str) -> None:
        '''Speichert die Gewichte des Netzwerks'''
        with open(file + '.npy', 'wb') as f:
            np.save(f,self.wih, allow_pickle=True)
            np.save(f,self.who, allow_pickle=True)
        print("Gewichte wurden gespeichert")            

    def load(self,file: str) -> None:
        '''Lädt die Gewichte des Netzwerks'''        
        with open(file + '.npy', 'rb') as f:
            self.wih = np.load(f)
            self.who = np.load(f)
        print("Gewichte wurden geladen")      
        
    def __str__(self) -> str:
        return "in -> hidden:" + np.array2string(self.wih) +"\nhidden -> out" + np.array2string(self.who)    

input_nodes = 4  # Vier verschiedene gemessenene Werte
hidden_nodes = 3
output_nodes = 3 # Drei Iris-Arten

knn = NeuralNetwork(input_nodes,hidden_nodes,output_nodes)
print(knn)
    

in -> hidden:[[-1.15820901  0.28184704 -0.00437989  0.93494344]
 [-0.18245714 -0.19688548 -0.36653645 -0.35199413]
 [-0.42456639  0.25163086  0.59019598  0.03031534]]
hidden -> out[[ 0.50583336 -0.84647608 -0.05806003]
 [ 0.23148974  0.46721303  0.60646773]
 [ 0.11700412 -0.42180946 -0.34364073]]


## Aufbau des Neuronalen Netzes
Unser künstliches neuronales Netz hat drei Schichten. Die Anzahl der ein- und Ausgabeknoten ist durch das Problem bestimmt: Wir haben vier Eingabewrte (Breite und Länge der zwei verschiedenen Blätter), und drei verschiedenen Iris-Arten. Bei der Anzahl der Knoten der mittleren Schicht sind wir frei. Hier mehr Knoten als in der Eingabeschicht zu verwenden, oder weniger Knoten als in der Ausgabeschicht, macht wenig Sinn. Im ersten Fall erhalten wir nicht mehr Information, im letzten Fall verlieren wir Information.

In [4]:
input_nodes = 4  # Vier verschiedene gemessenene Werte
hidden_nodes = 3
output_nodes = 3 # Drei Iris-Arten

knn = NeuralNetwork(input_nodes,hidden_nodes,output_nodes)

## Trainingsphase

In [5]:
def trainiere_knn(daten,epochen = 10):
    for e in range(epochen):
        np.random.shuffle(daten)
        for x,y in daten:
            inputs = np.asfarray(x)
            targets = np.zeros(output_nodes)  # Der Output wird auf 0 gesetzt...
            targets[y] = 1.                   #... bis auf das richtige Neuron, das auf 1 gesetzt wird
            knn.train(inputs, targets)       
            pass
        pass

trainiere_knn(trainings_daten,1000)   

### Abfrage des Neuronalen Netzwerks
Wir können nun unser neuronales Netzwerk abfragen.

In [6]:
def bestimme_iris(input):
    return ir.inv_mapping[knn.result(input[0])]


In [7]:
rindex = int(len(test_daten) * np.random.random())
print(test_daten[rindex] , '->', bestimme_iris(test_daten[rindex]),"(",test_daten[rindex][1],")")

[array([0.56410256, 0.41025641, 0.16666667, 0.02564103]), 0] -> Iris-setosa ( 0 )


## Test des Neuronalen Netzes anhand der Testdaten

Wie gut ist unser neuronales Netz eigentlich wirklich? Dazu zählen wir die Treffer auf der gesamten Menge der Testdaten:

In [8]:
def teste_knn(daten):
    tests = []    
    for x,y in daten:
        tests.append(1 if (knn.result(x) == y) else 0)
    return tests
 
tests = teste_knn(test_daten)    
print(tests)
print ("Testergebnis = ", np.floor(np.asarray(tests).sum() / len(tests) * 100) , "%")

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Testergebnis =  94.0 %


### Wir speichern die Gewichte des Netzwerks zur folgenden Analyse

In [9]:
knn.save("Iris")

Gewichte wurden gespeichert
